In [1]:
# Generic: A Modular Multi-Pipeline Framework for Probability Fusion Ensembles
# Specific: Cross-Lingual Sentiment Analysis with Probability Fusion Ensembles: A Modular Multi-Pipeline Framework for Low-Resource Languages

import os
import pandas as pd
import pprint

from pathlib import Path
from transformers import pipeline

from src.config import *
from src.metrics import evaluate_pipe
from src import (
    context,
    helper,
    sentiment, 
    utility, 
)

os.environ["MAMBA_USE_MAMBAPY"] = MambaBackend.FORCE_CUDA
os.environ["TOKENIZERS_PARALLELISM"] = "false"
helper.list_config()

if AppConfig.INFER:
    sample_texts = [
        "Maganda ang serbisyo at mabilis ang delivery!",
        "Sobrang pangit ng karanasan ko.",
        "It was okay, nothing special.",
    ]
    sentiment.infer(sample_texts, MambaConfig)
    sentiment.infer(sample_texts, MBertConfig)
else:
    if AppConfig.ENSEMBLE:
        temps  = [1.1, 0.9]     # example; learn these
        weights = [0.4, 0.6]    # example; learn these
        ens = sentiment.ensemble([MBertConfig, MambaConfig], temps, weights)
        print(ens)
    else:
        #mbert_context = context.setup_pipeline(MBertConfig, require_translation = False)
        #mbert_trainer = sentiment.train(mbert_context)

        xlmr_context = context.setup_pipeline(XlmrConfig, require_translation = False)
        xlmr_trainer = sentiment.train(xlmr_context)
        
        mamba_context = context.setup_pipeline(MambaConfig, require_translation = True)
        mamba_trainer = sentiment.train(mamba_context)



Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/69 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Precision Macro,Recall Macro
1,1.226500,1.146971,0.400000,0.190476,0.133333,0.333333
2,1.134700,1.143504,0.400000,0.190476,0.133333,0.333333
3,1.133000,1.106885,0.400000,0.190476,0.133333,0.333333
4,1.105700,1.102996,0.500000,0.371795,0.481481,0.444444
5,1.102000,1.105130,0.300000,0.166667,0.111111,0.333333


...

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Precision Macro,Recall Macro
1,1.360300,1.320498,0.200000,0.169312,0.138889,0.222222
2,1.112500,1.187799,0.200000,0.133333,0.095238,0.222222
3,0.907200,1.116316,0.500000,0.492063,0.500000,0.500000
4,0.767500,1.173165,0.400000,0.407143,0.455556,0.388889
5,0.690300,1.228606,0.500000,0.484127,0.616667,0.472222



### STATE-SPACES/MAMBA-130M-HF EVALUATION METRICS ###
{'epoch': 5.0,
 'eval_accuracy': 0.42857142857142855,
 'eval_f1_macro': 0.44230769230769235,
 'eval_loss': 0.9391617774963379,
 'eval_precision_macro': 0.42857142857142855,
 'eval_recall_macro': 0.46296296296296297,
 'eval_runtime': 0.1861,
 'eval_samples_per_second': 112.846,
 'eval_steps_per_second': 16.121}


Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/69 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/21 [00:00<?, ? examples/s]